<a href="https://colab.research.google.com/github/DINGMAN17/Learning_material/blob/main/3D_recon/3D_recon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from the textbook [Multiple View Geometry in computer Vision](https://books.google.com.sg/books/about/Multiple_View_Geometry_in_Computer_Visio.html?id=si3R3Pfa98QC&source=kp_book_description&redir_esc=y) and lecture notes [Standford CS231A ](http://web.stanford.edu/class/cs231a/);
content is available [on Github](https://github.com/DINGMAN17/Learning_material/tree/main/3D_recon).

**Please install the necessary packages to run the code**

In [ ]:
%pip install scipy numpy opencv-python tqdm glob

# Multi-view 3D reconstruction code demo

##Table of contents:
---
**1.   Camera basics**

**2.   Epipolar Geometry**


## 1 Camera basics

**1.1 Camera Matrix Model**

**1.2 Camera calibration**

**1.3 Camera Distortion**

**1.4 Code demo 1: Camera calibration for distortion correction**



### ***1.1 Camera Matrix Model***

Camera Matrix Model describes a set of important parameters that affect how a **world point $P$ is mapped to image coordinates $P'$**.










#### ***1.1.1 Intrinsic parameters***

*   Translation vector $[c_x\ c_y]^T$: describe how 
image plane and digital image coordinates can differ by a translation
*   Change of units $k$ and $l$: digital images and image plane are represented in different units, one in pixels and one in physical measurements

*   Skewness $\theta$: caused by sensor manufacturing errors
*   [Distortion](https://en.wikipedia.org/wiki/Distortion_%28optics%29) (ignore for now)

Make use of [Homogeneous coordinates](https://en.wikipedia.org/wiki/Homogeneous_coordinates#:~:text=Any%20point%20in%20the%20projective,multiplied%20by%20a%20common%20factor), a point in 3D space and its image coordinates by a matrix vector relationship can be expressed as:
$$P' =
\begin{bmatrix}
\alpha & -\alpha\cot\theta & c_x\\
0 & \frac{\beta}{\sin\theta} & c_y\\
0 & 0 & 1
\end{bmatrix}\begin{bmatrix}
I & 0\\
\end{bmatrix}P = K\begin{bmatrix}
I & 0\\
\end{bmatrix}P$$

The matrix $K$ is often referred to as the **camera matrix**
$$ K = \begin{bmatrix}
x'\\y'\\z\\
\end{bmatrix}=\begin{bmatrix}
\alpha & -\alpha\cot\theta & c_x\\
0 & \frac{\beta}{\sin\theta} & c_y\\
0 & 0 & 1
\end{bmatrix}$$

Camera matrix $K$ has **5 degrees of freedom**: 2 for focal length, 2 for offset, and 1 for skewness. (assume no distortion). $K$ is unique and inherent to a given camera.

#### ***1.1.2 Extrinsic Parameters***
The above mapping is between a point $P'$ in the **3D camera reference system** to a point $P$ in the **2D image plane**. However, the information about the 3D world may be available in a different coordinate system. Hence, additional transformation captured by **rotation matrix $R$** and **translation vector $T$** is introduced to relate points from the **world reference system** to the **3D camera reference system**

Given a point in a world reference system $P_w$, we can compute its camera coordinates:
$$P'=K\begin{bmatrix}
R & T\\
\end{bmatrix}P_w=MP_w$$

Matrices $R$ and $T$ are known as the **extrinsic parameters** as do not depend on the camera.

With extrinsic and intrinsic parameters $M$, mapping from a 3D point P in an arbitrary world
reference system to the image plane can hence be achieved.

In total, the projection matrix $M$ has **$11$ degrees of freedom**: 5 from the intrinsic camera matrix,
3 from extrinsic rotation, and 3 from extrinsic translation.


### ***1.2 Camera calibration***
**Purpose:** Estimate the extrinsic and intrinsic camera parameters.

**How:** Solve for the intrinsic camera matrix $K$ and
the extrinsic parameters $R, T$

**Setup:** Provide calibration rig which consists of a simple pattern, $i.e.$ checkerboard) with known dimensions. the rig defines our world reference frame with origin $O_w$ and axes $i_w,\ j_w,\ k_w$. From the rig's known pattern, we have known points
in the world reference frame $P_1,..., P_n$. Finding these points in the image we
take from the camera gives corresponding points in the image $p_1,..., p_n$.. 
<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/calib_setup.PNG' alt="Setup">
<figcaption>Figure1: The setup of an example calibration rig</figcaption></center>
</figure>

Hence, a linear system of equations from $n$ correspondences can be set up such that for each correspondence $P_i$, $p_i$ and camera matrix $M$ whose rows are $m_1,m_2,m_3$:

$$p_i=\begin{bmatrix}
u_i \\ v_i\\
\end{bmatrix}=MP_i=\begin{bmatrix}
\frac{m_1p_i}{m_3p_i} \\ \frac{m_2p_i}{m_3p_i}\\
\end{bmatrix}$$

which is equivalent of solving a pair of equations:
$$u_i(m_3P_i)-m_1P_i=0$$
$$v_i(m_3P_i)-m_2P_i=0$$

Here, there are $2$ constraints for solving the unknown parameters contained in $m$. From 1.1.2, we know that the **camera matrix $M$ has 11 unknown parameters**. This suggests that we need **at least $6$ correspondences** to solve this.

**Note:** Not all sets of $n$ correspondences will work. For example, if the points $Pi$ lie on the same plane, then the system will not be able to be solved. These unsolvable configurations of points are known as **degenerate configurations**.

In the real world applications (such as the demo in 1.4), we often use more than 6 points as measurements are often noisy. When $2n > 11$, the above homogeneous linear system is overdetermined. Therefore, it can be treated as a minimization problem and can be solved using [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition).

### **1.3 Camera distortion**

### **1.4 Code demo 1: Camera calibration for distortion correction**

This code demo is written in [Python](https://www.python.org/) which uses pre-defined functions from [OpenCV](https://opencv.org/) library. The reference can be found [here](https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html). 

The distorted images used in this demo were taken by a thermal-RGB camera for drone operations, the detail specs can be found [here](https://www.dji.com/sg/zenmuse-xt2).

<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon//sample_rgb_distort.jpg' alt="sample distorted rgb image">
<figcaption>Figure2: sample distorted rgb image</figcaption></center>
</figure>

**Interesting application:** It is possible to calibrate thermal images as well using a spcially fabricated chessboard made of aluminium foil. Unlike RGB images, pre-processing is required to extract the temperature and set thresholds for the thermal images. As shown in the two images below, raw thermal image is on the left and the processed image is on the right.

<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/sample_thermal_distort.JPG' width="425"/> <img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/sample_thermal_theshold.JPG' width="425"/> 





Import packages

In [ ]:
import os
import glob
import tqdm
import cv2 as cv
import numpy as np

In [ ]:
def get_camera_params(number_rows,number_cols,input_folder,print_params=False):
    '''
    get camera parameters based on a set of chessboard images (>10 images)

    Parameters
    ----------
    number_rows, number_cols : int, point number of rows/columns of chessboard
    input_folder : str
    print_params : boolean, optional
                  whether to print the camera parameters
    '''
    # findChessBoard flags
    find_flags = cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE + \
    			        cv.CALIB_CB_FILTER_QUADS + cv.CALIB_CB_FAST_CHECK
       
    # termination criteria 
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
       
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((number_cols*number_rows, 3), np.float32)
    objp[:,:2] = np.mgrid[0:number_rows, 0:number_cols].T.reshape(-1,2)
    
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    
    # read images
    images = glob.glob(os.path.join(input_folder, '*.JPG')) 
    fail_all = True

    for fname in tqdm(images):
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)    

        # Find the chess board corners
        ret,corners = cv.findChessboardCorners(gray,(number_rows,number_cols),
                                               find_flags)
    
        # If found, add object points, image points (after refining them)    
        if ret == True:
            fail_all = False
            objpoints.append(objp)
            # improve accuracy
            corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)
            
            # Draw the corners 
            cv.drawChessboardCorners(img,(number_rows,number_cols),corners2,ret)

            # save images with corner points
            raw_img = os.path.split(fname)     
            img_name = 'draw_'+raw_img[-1]   
            cv.imwrite(os.path.join(input_folder, img_name), img)
           
            # display succeed or fail to find corners
            print(fname, ': succeed to find corner')
        else:
            print(fname, ': fail')
        
    # Calibration camera to find intrinsic, extrinsic parameters
    if fail_all == False:
        imageSize = gray.shape[::-1]
        err,KK,distCoeffs,rvecs,tvecs = cv.calibrateCamera(objpoints,imgpoints,
                                                           imageSize,None,None)
        
        # Save the camera parameters
        parameter_arry = np.array([err, KK, distCoeffs, rvecs, tvecs])
        np.save('calibration_parameters.npy', parameter_arry)
        
        # # Print calibaration parameters
        if print_params:
            print('err', err)
            print('KK', KK)
            print('distCoeffs', distCoeffs)
            print('rvecs', rvecs)
            print('tvecs', tvecs)
        
        # Calculate re-projection error
        mean_error = 0
        for i in range(len(objpoints)):
            imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i],
                                             tvecs[i], KK, distCoeffs)
            error = cv.norm(imgpoints[i],imgpoints2,cv.NORM_L2)/len(imgpoints2)
            mean_error += error
        print("total error: ", mean_error/len(objpoints))
        error_list = [['mean_error',mean_error],['len(objpoints)',len(objpoints)],
                      ["total error: ", mean_error/len(objpoints)]]
        np.savetxt('Re_projection_error.csv',error_list,delimiter=',',fmt ='% s')
        return err, KK, distCoeffs, rvecs, tvecs
    else:
        return

## **2 Epipolar Geometry**

#### How is the Fundamental matrix useful:
if we know the Fundamental matrix, then simply knowing a point in an image
gives us an easy constraint of the corresponding point in the other image. 

Therefore, without knowing the actual position of $P$ in $3D$ space, or any of the extrinsic or intrinsic characteristics of the cameras, we
can establish a relationship between any $P$ and $P_0$.

Algorithm

*   The Eight-Point Algorithm
*   The Normalized Eight-Point Algorithm

<a id='8_point'></a>
### 2.1 The Eight-Point Algorithm

<a id='normalized_8'></a>
### 2.2 The Normalized Eight-Point Algorithm